**学生用**

# 2019年度知識情報演習III（後半）ノートブック 3
本ノートブックでは、情報検索システムにおける索引語の重み付け処理を理解しながら、Pythonでプログラムを作成していきます。

## 完了の目安
本ノートブックは、後半第3回（今回）の終わりか、第4回（次回）の前半までに完了することを目安にしましょう。

---
## ノートブック2の演習内容をTAか教員に確認してもらってから、ノートブック3に進んでください。

但し、確認の順番待ちの間に次のノートブックに取り掛かるのは構いません。

---
## 【重要】本演習の学習方針

プログラミングでは、ある問題を解決するための方法が複数存在するのが普通です。しかし本演習では、Pythonを使ったプログラミングにおいて**標準的な実装方法**を身につけることを重視します。したがって、以下の点に注意して演習を進めてください。

- プログラムを作成する際に、ノートブックで説明されたやり方に沿って実装すること
  - TAや教員もノートブックで説明した実装方法が実践できているかを確認します
- 外部ウェブサイトなどで示された別の実装方法を参照した場合、プログラムが正常に機能したとしても**やり直しをしてもらう場合がある**
- 情報検索の仕組みを理解するために、便利なライブラリを意図的に使わない場合がある
- 情報検索システムの理解に直接関係ない複雑な処理に関して、便利なライブラリを使用する場合がある
- その他の関連情報については、参照URLを示すので、授業外活動として各自学習しておくこと

---
## 【重要】本演習の実行環境

プログラミングは、プログラムを実行する環境によって動作や出力結果が異なる場合があります。全学計算機システムの実行環境は以下の通りです。

- OS : Ubuntu 16.04 LTS
- Python : Anaconda Python 3 ver 5.2.0
- JupyterLab : 0.32.1
- Janome : 0.36

研究室や個人のパソコンを使った場合、上記の実行環境と異なると同じ結果が出ない場合やエラーが出る（あるいは出ない）場合がありますので、注意が必要です。本演習では上記実行環境以外の動作に関するサポートは行いません。

---
## 重み付け

重み付け処理では、以下の手続きを順に行っていきます。

> 1. 重み付けの基礎と`tfidf`法の理解  
> 2. 索引ファイルの読み込みと`tf`値の取得  
> 3. `idf`値と`tfidf`値の算出  
> 4. 重みデータ付き索引ファイルの保存

---
### 演習用フォルダ構成の確認
作業を始める前に、演習用のフォルダが以下の構成になっていることを確認してください。  

```
ホームフォルダ
  ┗ CJE3
    ┗ ipynb
      ┣ CJE3_2019_Notebook_0.ipynb
      ┣ CJE3_2019_Notebook_1.ipynb
      ┣ CJE3_2019_Notebook_2.ipynb
      ┣ CJE3_2019_Notebook_3.ipynb（新規ファイル）
      ┗ images
          ┗ nested_dict.png
    ┗ data
      ┣ sample_doc1.txt
      ┣ ...
      ┗ sample_doc10.txt
    ┗ index
        ┗ index.txt
```

---
### 索引語の重み付けの基礎

前回までの索引付けの処理の結果、各文書に現れる語を切り出して、索引語として保存しました。これら文書には様々な索引語が含まれていますが、検索における**索引語の有用性**は均等ではありません。したがって、全ての索引語を同等に扱うと、検索結果の質が低くなってしまいます。

> 重み付けとは、文書に現れる有用な索引語には高いスコアを与え、そうでない索引語には低いスコアを与える処理です。こういった処理を行うことで、検索結果の質を向上します。ここでは、**重み＝有用性**と理解しておきましょう。英語では、`weight`を使います。

しかし、何をもって索引語の有用性を判断するのでしょうか？**情報検索**という研究分野では、過去何十年にもわたって、多くの研究者が様々な重み付けの手法を考案し、テストしてきました。

本演習では、その中でも基本中の基本である**`tfidf`法**を実装することで、重み付け処理の基礎を学んでいきます。

> `tfidf`法は情報検索のみならず、自然言語処理や情報推薦システムなどにも応用されている手法です。

---
### `tfidf`法とは

`tfidf`法は、

- `Term Frequency (tf)`：文書内の索引語の出現頻度（**索引語頻度**などと呼ばれます）
- `Inverse Document Frequency (idf)`：索引語が出現する文書数の逆数（**逆文書頻度**と呼ばれます）

という2種類の統計情報をもとにして、索引語を重み付けする手法です。

> と、言葉で表現しても、プログラムは書けません。

**アレルギーを発症する方もいるかもしれませんが**、`tf`、`idf`、そして両者の組み合わせである`tfidf`を順番に数式で見ていきましょう。この後に詳細な説明がありますので、まずは簡単に見るだけで構いません。

\begin{align}
tf(t, d) & = f_{t,d} \\
idf(t, D) & = \log(\frac{N}{n_t} + 1)\\
tfidf(t, d, D) & = f_{t,d} \times \log(\frac{N}{n_t} + 1)
\end{align}


#### `tf`の説明

\begin{align}
tf(t, d) & = f_{t,d} \\
\end{align}


左辺の$tf(t, d)$は、語`t`の文書`d`における出現頻度は、という意味です。

右辺の$f_{t, d}$は、語`t`が文書`d`に出現する頻度（回数）、という意味です。

> りんごはりんごです、と言っているようなものです。  
> つまり、`tf`は単なる出現回数です。

`tf`という重み付けでは、ある文書に出現する回数が多ければ多いほど、高いスコアを与えます。

また、ある索引語の`tf`値は、**各文書ごとに求める**ことに注意しましょう（下の表を参照）。

|索引語|文書|`tf`値|
|:--:|:--:|:--:|
|犬|文書 1|1|
|犬|文書 2|2|
|犬|文書 3|1|
|ネコ|文書 1|2|
|ネコ|文書 2|1|
|...|||

#### `idf`の説明

\begin{align}
idf(t, D) & = \log(\frac{N}{n_t} + 1)\\
\end{align}


左辺の$idf(t, D)$は、文書集合`D`における語`t`の逆文書頻度は、という意味です。

右辺の$\log\frac{N}{n_t}$は、文書集合`D`の総文書数`N`を語`t`が出現する文書数$n_t$で割ったもの（に`1`を足したもの）、の対数（$\log$）、という意味です。

> `tfidf`法では、対数の底は何でもいいので、特に明記しません。プログラムでも指定する必要はありません。  
> 対数をとる前に、値に`1`を足しているのは、全ての文書に出現する語があるとき、$\frac{N}{n_t}=1$になり、$\log(1)=0$になります。そうすると、`tf`の値がいくつでも、掛け算である`tfidf`の値が`0`になってしまいます。そのような状況を避けるために、便宜的に`1`を加算して、$\log(1)$を回避しているのです。

`idf`という重み付けでは、ある索引語が出現する文書数が少なければ少ないほど、高いスコアを与えます。

`idf`で注意すべきは、文書集合内において語が出現する**文書の数**を数えているのであって、`tf`値のように各文書の中に何回出現するかは考慮しない点です。

また、`idf`値は、各索引語に一つだけ与えられることに注意しましょう（下の表を参照）。

|索引語|総文書数|出現文書数|`idf`値|
|:--:|:--:|:--:|:--:|
|犬|10|3|1.466|
|ネコ|10|2|1.791|
|www|10|10|0.693|


#### `tfidf`の説明

\begin{align}
tfidf(t, d, D) & = f_{t,d} \times \log(\frac{N}{n_t}+1)
\end{align}

左辺の$tfidf(t, d, D)$は、文書集合`D`に含まれる文書`d`における語`t`の`tfidf`の値は、という意味です。

右辺の$f_{t,d} \times \log(\frac{N}{n_t}+1)$は、$f_{t,d}$と$\log(\frac{N}{n_t}+1)$を掛けたもの、という意味です。

また、ある索引語の`tfidf`値は、`tf`値同様に、各文書ごとに求めることに注意しましょう（下の表を参照）。

|索引語|文書|`tf`値|`idf`値|`tfidf`値|
|:--:|:--:|:--:|:--:|:--:|
|犬|文書 1|1|1.466|1.466|
|犬|文書 2|2|1.466|2.932|
|犬|文書 3|1|1.466|1.466|
|ネコ|文書 1|2|1.791|3.582|
|ネコ|文書 2|1|1.791|1.791|


上の表の`tfidf`値が、`tfidf`法を用いて算出した、各索引語の各文書における重み（＝有用性の大きさ）です。

---
### tf値の求め方

それでは、`tf`値、`idf`値、`tfidf`値を計算するプログラムを順に実装していきましょう。

まずは、`tf`値ですが、これは前回までの索引付けプログラムの中で既に取得済みなので、新たに計算する必要はありません。その代わり、前回のプログラムでファイルに保存した索引ファイルを開き、データを辞書オブジェクトに取り込むプログラムを書きましょう。

保存した索引ファイルは以下のような構成でした。各要素は不可視のタブ文字（`\t`）で区分されています。

```
あっ	sample_doc2.txt	1
あっ	sample_doc3.txt	1
あっ	sample_doc4.txt	1
あと	sample_doc2.txt	1
あまりに	sample_doc4.txt	1
...
```

---
#### 文字列の分割

ある文字列を区切り文字を使って分割するには、`split`メソッドを使います。

In [1]:
# 文字列の分割
line = "あっ	sample_doc2.txt	1"
split_line = line.split("\t")

# 確認用出力
split_line

['あっ', 'sample_doc2.txt', '1']

---
それでは、上のプログラムを実行してみましょう。

> `['あっ', 'sample_doc2.txt', '1']`

と出力されたら成功です。

`line`変数に代入されたタブ文字で区切られた文字列が、`split`メソッドによって区分され、その結果が、`split_line`という配列変数に代入されました。区切り文字に使う文字は、`.split(...)`の`...`の部分で指定されています。

`split`メソッドは、分割をしたい文字列変数名（上記の例だと`line`）の後に、`.`を使って接続して使います。

---
#### 配列要素の取得と辞書オブジェクトへの代入
索引ファイルのデータ行を分割できたので、出力された配列の各要素を使って辞書オブジェクトに代入していきます。Notebook2で実装したように、索引ファイルの辞書オブジェクトは、索引語とファイル名の二つのキーを使う構造です。

In [2]:
# 文字列の分割
line = "あっ	sample_doc2.txt	1"
split_line = line.split("\t")
print(split_line)
# 辞書オブジェクトの初期化
dict = {}

# 索引ファイルデータの代入
word = split_line[0] # 索引語
doc = split_line[1] # ファイル名
freq = int(split_line[2]) # 出現頻度（整数型にキャスト）

if word in dict:
    dict[word][doc] = freq
else:
    dict[word] = {}
    dict[word][doc] = freq

# 確認用出力
dict

['あっ', 'sample_doc2.txt', '1']


{'あっ': {'sample_doc2.txt': 1}}

---
上のプログラムを実行してみましょう。

> `{'あっ': {'sample_doc2.txt': 1}}`

と出力されたら、分割した配列要素が辞書オブジェクトに代入されています。

- 9-11行目：分割された配列要素を**わかりやすい名前の変数**に置き換えています
- 12行目：索引語をキーとするレコートの中にもう一つの辞書オブジェクトを作成しています
- 13-17行目：辞書オブジェクトにキーが存在するかを確認しながら、ファイル名と出現頻度を代入しています

#### 変数の型変換（キャスト）

さらに、11行目に注意してください。

テキストファイルから読み込むデータは**文字列**として扱われます。しかし、後でも出てきますが、頻度データは**数値**です。人間にとってはどちらも`1`や`2`など見た目は同じですが、`python`にとっては、大きな違いなのです。したがって、ファイルからデータを読み込む時に文字列型になっている変数を、辞書オブジェクトに代入する前に、整数型に変換しておきます。

> このような変数の型を変換することを**キャスト**するとも呼びます。  
> 整数型に変換されたことは、配列の出力結果が、`'1'`ではなく`1`と`'`がないことから判断できます。

後ほど、変数の型に関する追加の説明がありますので、そちらも合わせて確認してください。

---
それでは実際に、以下の作業をおこなうプログラムを作成しましょう。

- 前回作成した索引ファイルを開く
- 開いたファイルを1行づつ読み込む
- 読み込んだ行を分割して配列に代入する
- 分割した配列要素を使って辞書オブジェクトにデータを代入する
- （確認のため）全体を**ソートして**出力する

---
#### 行末の改行文字の削除

ファイルからデータを読み込む時に気をつけなければならないのは、各行の末尾に改行コード（`\n`）が付いていることです。したがって、`for`ループを使ってファイル内のデータを行ごとに処理する場合は、

> `line = line.rstrip()`

のように、`.rstrip()`メソッドを使って、対象となる文字列（上記の例では`line`）の末尾にある改行文字を削除する必要があります。

この処理を`for`ループの直後に使いましょう。

---

In [3]:
# 必要なライブラリを読み込む
import os
from janome.tokenizer import Tokenizer
import re
t = Tokenizer()

# 索引ファイルの指定
f_name = "../index/index.txt"


# 辞書オブジェクトの初期化
dict = {}

# 索引ファイルを開く
f = open(f_name)

# 索引ファイルを1行づつ処理
for line in f:
    # 行末の改行文字除去
    line = line.rstrip()
    # 行の分割
    split_line = line.split("\t")
    #print(split_line)
    # 配列要素
    word = split_line[0] # 索引語
    doc = split_line[1] # ファイル名
    freq = int(split_line[2]) # 出現頻度（整数型にキャスト）

    # 辞書オブジェクトへの代入
    if word in dict:
        dict[word][doc] = freq
    else:
        dict[word] = {}
        dict[word][doc] = freq
        
# 全体の出力
print(dict)



{'あっ': {'sample_doc2.txt': 1, 'sample_doc3.txt': 1, 'sample_doc4.txt': 1}, 'あと': {'sample_doc2.txt': 1, 'sample_doc6.txt': 1}, 'あまりに': {'sample_doc4.txt': 1}, 'ある': {'sample_doc1.txt': 1, 'sample_doc10.txt': 1, 'sample_doc2.txt': 1, 'sample_doc3.txt': 1, 'sample_doc5.txt': 1, 'sample_doc7.txt': 1, 'sample_doc8.txt': 1, 'sample_doc9.txt': 2}, 'あるい': {'sample_doc10.txt': 1}, 'あろ': {'sample_doc3.txt': 1}, 'いか': {'sample_doc10.txt': 1}, 'いくら': {'sample_doc6.txt': 1}, 'いた事': {'sample_doc1.txt': 1}, 'いっ': {'sample_doc10.txt': 1, 'sample_doc9.txt': 1}, 'いとも': {'sample_doc2.txt': 1}, 'いる': {'sample_doc1.txt': 1, 'sample_doc10.txt': 1, 'sample_doc3.txt': 1, 'sample_doc4.txt': 1, 'sample_doc6.txt': 2}, 'いわゆる': {'sample_doc3.txt': 1}, 'うち': {'sample_doc10.txt': 1, 'sample_doc6.txt': 1}, 'おかしい': {'sample_doc7.txt': 1}, 'おっ': {'sample_doc6.txt': 1, 'sample_doc7.txt': 1}, 'かかる': {'sample_doc9.txt': 1}, 'かた': {'sample_doc4.txt': 1}, 'かも': {'sample_doc10.txt': 1}, 'から': {'sample_doc10.txt': 1, 'sample

---
読み込んだ索引ファイルと同じ内容のデータが出力されたら成功です。

以上で、索引ファイルの読み込みと`tf`値の取得ができました。

---
### `idf`値の取得

それでは、次に、作成した辞書オブジェクトを使って`idf`値を取得してきましょう。

現在、辞書オブジェクト`dict`の中は以下のような構成になっています（詳細は講義スライドを参照してください）。

|キー：索引語|値：子オブジェクトへのリンク|
|:--:|:--|
|あっ| → |
|あと| → |
|ある| → |

**あっ**の入れ子辞書オブジェクト

|キー：ファイル名|値：出現頻度|
|:--:|:--:|
|sample_doc2.txt|1|

**あと**の入れ子辞書オブジェクト

|キー：ファイル名|値：出現頻度|
|:--:|:--:|
|sample_doc2.txt|1|

**ある**の入れ子辞書オブジェクト

|キー：ファイル名|値：出現頻度|
|:--:|:--:|
|sample_doc1.txt|1|
|sample_doc2.txt|1|


また、`idf`の式を見ると、

- $N$: 総文書数
- $n_t$: ある索引語$t$の出現文書数

の二つの数値が必要なことがわかります。

`dict`から、これらの数値を順番に取得していきましょう。

> 両方の数値取得に共通する処理は、個別の辞書オブジェクトの作成とそのレコード数の取得、です。

---
#### 個別の辞書オブジェクトの作成（総文書用）

辞書オブジェクト`dict`に含まれる文書数は、入れ子辞書オブジェクトのファイル名を走査することで算出できます。そこで、

- `dict`内のファイル名を保存するための辞書オブジェクト`docs`を新規作成
- `dict`内の子オブジェクトを順番に処理して、ファイル名を`docs`に追加

という処理をおこなうプログラムを考えてみましょう（実行はできません）。

In [4]:
# ファイル名保存用辞書オブジェクト
docs = {}

# dictを一つずつ処理
for word in dict:
    # 入れ子を一つずつ処理
    for doc in dict[word]:
        # もし辞書オブジェクトにファイル名がないなら追加
        if doc not in docs:
            docs[doc] = 1

---
プログラムの説明

> 2: ファイル名保存用辞書オブジェクトの初期化  
> 5: dict内のキー（索引語）を一つずつ処理する  
> 7: 索引語キーが指す入れ子オブジェクト内のキー（ファイル名）を一つずつ処理する  
> 9: `not in`メソッドを使って、変数`doc`がもつファイル名が、辞書オブジェクト`docs`に存在しない時に真となる条件文  
> 10: 9で真となった（＝未知の）ファイル名をキーとしたレコードを`docs`に追加する（値は適当で良い）


ご覧の通り、手続きそのものは索引語の出現頻度を数えた時と似ています。ポイントは、9行目で`not in`メソッドを使って、辞書オブジェクトにキーが**存在しない**時に真を返す条件文を作成することで、プログラムをすっきりさせている点です。

---
#### 辞書オブジェクトのレコード数

上記の処理で、`dict`内の全てのファイル名が`docs`に格納されたので、`docs`のレコード数を取得すれば、`idf`式の$N$が求まります。

辞書オブジェクトのもつレコード数は、以下のように`len`メソッドを使って取得します。

In [5]:
# サンプル辞書オブジェクトの生成（キーと値は適当）
sample_dict = {}
sample_dict['item1'] = 1
sample_dict['item2'] = 1
sample_dict['item3'] = 1

# 辞書オブジェクトのレコード数の取得
sample_dict_size = len(sample_dict)

# 確認用出力
print('レコード数：' + str(sample_dict_size))

レコード数：3


---

プログラムの説明

> 2: 辞書オブジェクト`sample_dict`の初期化  
> 3-5: サンプルデータの代入  
> 8: `sample_dict`のレコード数を`len`メソッドで取得し、変数`sample_dict_size`に代入  
> 11: 確認の出力

それでは、上のプログラムを実行してみましょう。

> `レコード数：3`

と出力されたら成功です。サンプルデータの数を増やして、出力結果が変化するか試してみましょう。

---
それでは、ここまで学んだことを使って、先ほど作成したプログラムに、総文書数を求める処理を追加してみましょう。

In [6]:
# 必要なライブラリを読み込む
import os
from janome.tokenizer import Tokenizer
import re
t = Tokenizer()

# 索引ファイルの指定
f_name = "../index/index.txt"


# 辞書オブジェクトの初期化
dict = {}

# 索引ファイルを開く
f = open(f_name)

# 索引ファイルを1行づつ処理
for line in f:
    # 行末の改行文字除去
    line = line.rstrip()
    # 行の分割
    split_line = line.split("\t")
    #print(split_line)
    # 配列要素
    word = split_line[0] # 索引語
    doc = split_line[1] # ファイル名
    freq = int(split_line[2]) # 出現頻度（整数型にキャスト）

    # 辞書オブジェクトへの代入
    if word in dict:
        dict[word][doc] = freq
    else:
        dict[word] = {}
        dict[word][doc] = freq

# ファイル名保存用辞書オブジェクト
docs = {}

# dictを一つずつ処理
for word in dict:
    # 入れ子を一つずつ処理
    for doc in dict[word]:
        # もし辞書オブジェクトにファイル名がないなら追加
        if doc not in docs:
            docs[doc] = 1
            
# docsのレコード数を求める
docs_size = len(docs)

# 確認用出力
print("総文書数:", docs_size)

総文書数: 10


---

> 総文書数：10

と出力されたら成功です。

---
#### 個別の辞書オブジェクトの作成（文書頻度用）

`dict`内の総文書数$N$を得ることができたので、次に索引語の文書頻度$n_t$を取得しましょう。ここでは、文書頻度の英語訳である`document frequency`の頭文字をとった`df`という辞書オブジェクトを新規作成します。

`df`のキーは索引語、値は文書頻度です。

索引語が出現する文書の数（＝文書頻度）はどのようにすれば取得できるでしょうか。冒頭の`dict`と入子オブジェクトの構造をもう一度よく見てください。そうすると、各索引語の文書頻度は、索引語がもつ入子辞書オブジェクトのレコード数と同様であることが分かります。

したがって、これまで何ども実装してきたように、`dict`内の要素を`for`ループで巡回し、各索引語の子オブジェクトのレコード数を`len`メソッドを使って取得すればよいことになります。そして、取得したレコード数を、新規作成した`df`に代入していきます。

---
#### 入れ子辞書オブジェクトのレコード数

上の例で辞書オブジェクトのレコード数を取得する方法がわかりましたが、索引データの辞書オブジェクトは入れ子構造になっています。したがって、`len`メソッドも入れ子の辞書オブジェクトに対して使う必要があります。

- 親オブジェクトの場合：`len(dict)`
- 子オブジェクトの場合：`len(dict[word])`

---
それでは、先ほどのプログラムに

- 索引語の`df`値を取得し、新しい辞書オブジェクト`df`に代入する
- （確認のために）辞書オブジェクト`df`を出力する

プログラムを追加してみましょう。

> 総文書数の確認用出力は不要です。

In [7]:
# 必要なライブラリを読み込む
import os
from janome.tokenizer import Tokenizer
import re
t = Tokenizer()

# 索引ファイルの指定
f_name = "../index/index.txt"


# 辞書オブジェクトの初期化
dict = {}

# 索引ファイルを開く
f = open(f_name)

# 索引ファイルを1行づつ処理
for line in f:
    # 行末の改行文字除去
    line = line.rstrip()
    # 行の分割
    split_line = line.split("\t")
    #print(split_line)
    # 配列要素
    word = split_line[0] # 索引語
    doc = split_line[1] # ファイル名
    freq = int(split_line[2]) # 出現頻度（整数型にキャスト）

    # 辞書オブジェクトへの代入
    if word in dict:
        dict[word][doc] = freq
    else:
        dict[word] = {}
        dict[word][doc] = freq

# ファイル名保存用辞書オブジェクト
docs = {}

# dictを一つずつ処理
for word in dict:
    # 入れ子を一つずつ処理
    for doc in dict[word]:
        # もし辞書オブジェクトにファイル名がないなら追加
        if doc not in docs:
            docs[doc] = 1
            
# docsのレコード数を求める
docs_size = len(docs)

# df用辞書オブジェクトの初期化
df = {}

# dictのキー（索引語）を一つずつ処理
for key in dict:
    # 入れ子辞書オブジェクトのレコード数を取得し、dfに代入
    df[key] = len(dict[key])

# 確認の出力
for key in sorted(df):
    print(key + '\t' + str(df[key]))


あっ	3
あと	2
あまりに	1
ある	8
あるい	1
あろ	1
いか	1
いくら	1
いた事	1
いっ	2
いとも	1
いる	5
いわゆる	1
うち	2
おかしい	1
おっ	2
かかる	1
かた	1
かも	1
から	6
かん	1
くい	1
くずれ	1
くらい	1
くれる	1
くわし	1
けむり	1
こう	1
ここ	2
この	5
これ	2
これから	1
こん	1
こんな	1
さえ	1
さて	1
さらさら	1
さり	1
しかし	1
しかも	1
しき	1
しばらく	2
しまっ	1
じめじめ	1
する	2
そう	2
そうして	1
そこ	1
その	4
そのうち	1
その後	1
それ	2
そろ	1
そろりと	1
たく	1
たくさん	1
ただ	1
たばこ	1
たら	2
だいぶ	1
だけ	2
ついに	1
つか	1
つかまえ	1
つるつる	1
でも	3
とある	1
とうてい	1
として	1
とにかく	1
とんと	1
どう	1
どうして	1
どうにか	1
どうも	2
どこ	1
ない	6
なかっ	2
なく	1
なっ	1
なら	2
なり	1
なる	3
のそのそ	1
のみ	1
はい出し	1
はじめ	1
はず	1
はっ	1
はてな	1
ばかり	1
ひだり	1
ひらに	1
ぴき	1
ふと	1
ふん	1
ぷうぷうと	1
べき	1
べつ	1
ぼう	1
ぽく	1
また	1
まだ	1
まで	4
まるで	1
むせ	1
むやみ	2
もう	1
もぐり込ん	1
もし	1
もの	5
ものの	1
やっ	1
やら	1
ゆう	1
よい	2
よう	1
ようやく	3
よかろ	1
よく	1
られ	2
わに	1
わら	1
をもって	1
スー	1
ニャー	1
ニャーニャー	1
フワフワ	1
一	1
一刻	1
一樹	1
一番	1
一疋	1
三	1
上	3
上今	1
不思議	1
中	3
事	4
云	1
人間	5
今	1
今日	1
仕方	2
付い	1
何	5
何だか	1
何となく	1
兄弟	1
先	1
内	1
出	3
出会	1
出来	1
分ら	2
分別	1
別に	1
別段	1
到底	1
前	1
助から	1
動く	1
名前	1
向う	1
吹く	1
吾輩	4
咽	1
善い	1
坐っ	2
垣根	1
声	1
大きな	1
妙	1
始	1
始め	3
始末	1
姿	1
実に	1
容子	1
寒	1
寒し	1
少し	1
崩	1
左	1
度	1
廻り	

---

> `あっ	3`  
> `あと	2`  
> `あまりに	1`  
> `ある	8`  
> `あるい	1`  
> ...

という出力が得られたら成功です。

---

それでは、最後に、`idf`の式に基づいて、各索引語の`idf`値を算出しましょう。

\begin{align}
idf(t, D) & = \log(\frac{N}{n_t}+1) \\
\end{align}

ここまでのプログラムで、

- 総文書数$N$は、`docs_size`
- 文書頻度$n_t$は、`df`

に格納されています。

これらを使って`idf`値を格納する辞書オブジェクト`idf`を新規作成しましょう。

辞書オブジェクト`idf`のキーは索引語、値は`idf`値です。

対数の計算は、`log`メソッドを用います。また、`log`メソッドを用いるには、

> import math

として、`math`ライブラリを読み込む必要があります。

In [8]:
# 必要なライブラリを読み込む
import math

# idf値用辞書オブジェクト
idf = {}

# dfのキー（索引語）を一つずつ処理
for word in df:
    # idf値を計算し、格納
    idf[word] = math.log( ( docs_size / df[word] ) + 1 )

# 確認の出力
for word in sorted(idf):
    print(word + '\t' + str(idf[word]))

あっ	1.4663370687934272
あと	1.791759469228055
あまりに	2.3978952727983707
ある	0.8109302162163288
あるい	2.3978952727983707
あろ	2.3978952727983707
いか	2.3978952727983707
いくら	2.3978952727983707
いた事	2.3978952727983707
いっ	1.791759469228055
いとも	2.3978952727983707
いる	1.0986122886681098
いわゆる	2.3978952727983707
うち	1.791759469228055
おかしい	2.3978952727983707
おっ	1.791759469228055
かかる	2.3978952727983707
かた	2.3978952727983707
かも	2.3978952727983707
から	0.9808292530117263
かん	2.3978952727983707
くい	2.3978952727983707
くずれ	2.3978952727983707
くらい	2.3978952727983707
くれる	2.3978952727983707
くわし	2.3978952727983707
けむり	2.3978952727983707
こう	2.3978952727983707
ここ	1.791759469228055
この	1.0986122886681098
これ	1.791759469228055
これから	2.3978952727983707
こん	2.3978952727983707
こんな	2.3978952727983707
さえ	2.3978952727983707
さて	2.3978952727983707
さらさら	2.3978952727983707
さり	2.3978952727983707
しかし	2.3978952727983707
しかも	2.3978952727983707
しき	2.3978952727983707
しばらく	1.791759469228055
しまっ	2.3978952727983707
じめじめ	2.3978952727983707
する	1.791759

---

> `あっ	1.4663370687934272`  
> `あと	1.791759469228055`  
> `あまりに	2.3978952727983707`  
> `ある	0.8109302162163288`  
> `あるい	2.3978952727983707`  
> ...

という出力が得られたら成功です。

---
#### Pythonにおける変数の型

ここで、`python`における変数の型について整理しておきましょう。

本演習で用いる型は以下の通りです。

|型|性質|例|キャストメソッド|
|:--:|:--:|:--:|:--:|
|string|文字列|`あいうえお`|`str(...)`|
|int|整数|`1234`|`int(...)`|
|float|浮動小数|`0.123`|`float(...)`|

ちなみに、上記で算出した`idf`値の型は、`float`です。

これ以外にも多くの型が存在します。興味のある方は、[本家のウェブサイト](https://docs.python.jp/3/library/stdtypes.html)を参照してください。

> 既に説明したように、例えば`print(...)`メソッド内では、`int`型や`float`型を直接使用することができないので、`string`型に一度キャスト（型変換）する必要があります。

> また、同じ数値であれば、自動的に変数がキャストされる場合があります。
> 例えば、次項でおこなう`tfidf`値の算出では、`int`型の`tf`値と`float`型の`idf`値を掛け算しています。この場合、両者の型は異なりますが、`int`型の`tf`値が`float`型に自動的にキャストされ、演算結果は、`float`型になります。

---
### `tfidf`値の算出

\begin{align}
tfidf(t, d, D) & = f_{t,d} \times \log(\frac{N}{n_t}+1)
\end{align}

ここまでの処理で、`tf`値を`idf`値を取得できましたので、これらを使って`tfidf`値を算出しましょう。`tfidf`の式からわかる通り、`tf`値と`idf`値をかけるだけです。

> `tf`値は`dict`の値として格納されていることに注意してください。

In [9]:
# dictを一つずつ処理
for word in sorted(dict):
    # 入れ子を一つずつ処理
    for doc in sorted(dict[word]):
        # tfidf値の算出
        tfidf = dict[word][doc] * idf[word]
        # 確認の出力
        print(word + '\t' + doc + '\t' + str(tfidf))

あっ	sample_doc2.txt	1.4663370687934272
あっ	sample_doc3.txt	1.4663370687934272
あっ	sample_doc4.txt	1.4663370687934272
あと	sample_doc2.txt	1.791759469228055
あと	sample_doc6.txt	1.791759469228055
あまりに	sample_doc4.txt	2.3978952727983707
ある	sample_doc1.txt	0.8109302162163288
ある	sample_doc10.txt	0.8109302162163288
ある	sample_doc2.txt	0.8109302162163288
ある	sample_doc3.txt	0.8109302162163288
ある	sample_doc5.txt	0.8109302162163288
ある	sample_doc7.txt	0.8109302162163288
ある	sample_doc8.txt	0.8109302162163288
ある	sample_doc9.txt	1.6218604324326575
あるい	sample_doc10.txt	2.3978952727983707
あろ	sample_doc3.txt	2.3978952727983707
いか	sample_doc10.txt	2.3978952727983707
いくら	sample_doc6.txt	2.3978952727983707
いた事	sample_doc1.txt	2.3978952727983707
いっ	sample_doc10.txt	1.791759469228055
いっ	sample_doc9.txt	1.791759469228055
いとも	sample_doc2.txt	2.3978952727983707
いる	sample_doc1.txt	1.0986122886681098
いる	sample_doc10.txt	1.0986122886681098
いる	sample_doc3.txt	1.0986122886681098
いる	sample_doc4.txt	1.0986122886681098
いる	sa

---

> `あっ	sample_doc2.txt	1.4663370687934272`  
> `あっ	sample_doc3.txt	1.4663370687934272`  
> `あっ	sample_doc4.txt	1.4663370687934272`  
> `あと	sample_doc2.txt	1.791759469228055`  
> `あと	sample_doc6.txt	1.791759469228055`  
> ...

という出力が得られたら成功です。

---
### 重み付き索引ファイルの保存

最後に、重み情報を追加した索引ファイルを新規に保存するプログラムを書きましょう。

書き出す新しいファイル名は、`index2.txt`です。

ファイルの出力フォーマットは

> 索引語　ファイル名　tf値　idf値　tfidf値

の5つの要素から成り、各要素はタブ文字（`\t`）で区切ってください。

**ソートするのを忘れずに**

In [10]:
# 必要なライブラリを読み込む
import math
# 索引ファイルの指定
f_name = "../index/index.txt"
f2_name = "../index/index2.txt"

# 辞書オブジェクトの初期化
dict = {}

# 読み出し用索引ファイルを開く
f = open(f_name)

# 書き出し用索引ファイルを開く
f2 = open(f2_name, 'w')

# 索引ファイルを1行づつ処理
for line in f:
    # 行末の改行文字除去
    line = line.rstrip()
    # 行の分割
    split_line = line.split("\t")
    
    # 配列要素
    word = split_line[0] 
    doc = split_line[1] 
    freq = int(split_line[2]) 

    # 辞書オブジェクトへの代入
    if word in dict:
        dict[word][doc] = freq
    else:
        dict[word] = {}
        dict[word][doc] = freq

# ファイル名保存用辞書オブジェクト
docs = {}

# dictを一つずつ処理
for word in dict:
    # 入れ子を一つずつ処理
    for doc in dict[word]:
        # もし辞書オブジェクトにファイル名がないなら追加
        if doc not in docs:
            docs[doc] = 1
            
# docsのレコード数を求める
docs_size = len(docs)

# df用辞書オブジェクトの初期化
df = {}

# dictのキー（索引語）を一つずつ処理
for key in dict:
    # 入れ子辞書オブジェクトのレコード数を取得し、dfに代入
    df[key] = len(dict[key])

# idf値用辞書オブジェクト
idf = {}

# dfのキー（索引語）を一つずつ処理
for word in df:
    # idf値を計算し、格納
    idf[word] = math.log( ( docs_size / df[word] ) + 1 )

# dictを一つずつ処理
for word in sorted(dict):
    # 入れ子を一つずつ処理
    for doc in sorted(dict[word]):
        # tfidf値の算出
        tfidf = dict[word][doc] * idf[word]
        # 重み付き索引データの出力
        f2.write(word + '\t' + doc +'\t' + str(dict[word][doc]) +'\t' + str(idf[word]) + '\t' + str(tfidf) + '\n')

# ファイルを閉じる
f.close()
f2.close()

---
**エラーが出力されずに**、上の入手力ブロックが緑色から青色に変化したら処理が終了した印です。画面には特に何も表示されません。

`~/CJE3/index/`フォルダに、`index2.txt`というファイルが作成され、その中に、重み情報が追加された索引語一覧が書き込まれているか確認しましょう。

出力例

```
あっ    sample_doc2.txt 1       1.4663370687934272      1.4663370687934272
あっ    sample_doc3.txt 1       1.4663370687934272      1.4663370687934272
あっ    sample_doc4.txt 1       1.4663370687934272      1.4663370687934272
あと    sample_doc2.txt 1       1.791759469228055       1.791759469228055
あと    sample_doc6.txt 1       1.791759469228055       1.791759469228055
あまりに        sample_doc4.txt 1       2.3978952727983707      2.3978952727983707
...
```

---
### プログラムの説明文を書く

それでは、本Notebookの最後の演習として、索引ファイルを生成するプログラムの説明文を書きましょう。説明文を書く上での注意点は前回のNotebookを参照してください。


> 2: 数学処理を扱うmathライブラリをインポート

> 4: 読み出し用索引ファイルの指定 

> 5: 書き出し用索引ファイルの指定

> 8: 辞書オブジェクトの初期化

> 11: 読み出し用ファイルを開く

> 14: 書き出し用ファイルを開く

> 17: 索引ファイルを１行ずつ変数lineに格納し最後まで繰り返す

> 19: lineの改行文字削除

> 21: lineをタブ文字で要素ごとに分けたものを配列split_lineに格納

> 24~26: 配列の要素を指定

> 29: もしdictにwordが含まれていたら

> 30: そのwordにファイル名と出現頻度を代入

> 31: 29行目がfalseならば

> 32: wordをキーとする無名辞書オブジェクトを初期化

> 33: ファイル名と出現頻度を代入

> 36: ファイル名保存用辞書オブジェクトを初期化

> 39: dictの内容を１つずつ取り出し最後の要素まで繰り返す

> 41: 索引語キーが指す入れ子オブジェクト内のキー（ファイル名）を一つずつ処理する

> 43: もし取り出したファイル名がdocsに存在しなかった場合

> 44: docsに追加

> 47: docsの要素数をlenメソッドを用いて求める

> 50: df算出用の辞書オブジェクトを初期化

> 53: dictのキーを１つずつ取り出す

> 55: 入れ子辞書オブジェクトのレコード数を取得し, dfに保存する

> 58: idf算出用の辞書オブジェクトを初期化

> 61: dfのキーを変数wordとして１つずつ取り出し最後の要素まで繰り返す

> 63: 定義に基づいてidf値を計算しidfに保存する

> 66: ソートしたdictのキーを変数wordとして１つずつ取り出し最後まで繰り返す

> 68: 入れ子辞書オブジェクトのキーを変数docとして１つずつ取り出し最後まで繰り返す

> 70: 定義に基づいてtfidfの値を算出する

> 72: 出力用ファイルに書き出す

> 75,76: ファイルを閉じる



---
## ここまで完成したら、TAか教員に確認してもらってから、ノートブック4にいきましょう。

### チェックポイント

- プログラムが期待通りに動作するか、索引ファイルは作成されているか
- ノートブックで説明したやり方に沿ってプログラムが書かれているか（我流で書いていないか）
- `math`が何をするために必要なライブラリか説明できるか
- 23行目の`split_line = line.split("\t")`の処理をデータ構造も含めて正確に説明できるか
- 28行目の`freq = int(split_line[2])`のキャスト処理を正確に説明できるか
- `log`が何をするためのメソッドか説明できるか
- プログラム末尾に`f.close()`と`f2.close()`を忘れていないか
- 説明文にライブラリ・クラス・変数・メソッド・ループなどのキーワードを正確に使用しているか

---
Copyright &copy; 2019. Hideo Joho and Haitao Yu. All rights reserved.

無断複製・転載・配布行為を禁止します。